In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import tensorflow as tf
from tensorflow import keras
import kapre
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Resizing, Conv2D
from bfm import BFMGenerator
import librosa
import torch
import random

# Preset Variables if needed:
RANDOM_STATE = 1
np.random.seed(RANDOM_STATE)

PROJECT_PATH = "/app/project"

In [2]:
SAMPLE_RATE     = 16_000
AUDIO_LENGTH_S  = 3
SPEC_HOPSIZE    = 256
SPEC_BLOCKSIZE  = 1024

NUM_SAMPLES = SAMPLE_RATE * AUDIO_LENGTH_S
TIME_STEPS = (NUM_SAMPLES - SPEC_BLOCKSIZE) // SPEC_HOPSIZE + 1
N_MELS = 128

In [3]:
def load_audio(path: str):
    y, sr = librosa.load(path, mono=True, sr=SAMPLE_RATE)
    if len(y) > NUM_SAMPLES:
        y = librosa.effects.trim(y)[0]
    if len(y) > NUM_SAMPLES:
        y = y[:NUM_SAMPLES]
    elif len(y) < NUM_SAMPLES:
        padding = NUM_SAMPLES - len(y)
        pad_left = padding // 2
        pad_right = padding - pad_left
        y = np.pad(y, (pad_left, pad_right), mode='constant')
    return y

In [4]:
df_path = os.path.join(PROJECT_PATH, "datasets", "test_005_ids.feather")
test_df = pd.read_feather(df_path)
test_df = test_df.dropna()

ID_COLUMN_NAMES = [c for c in test_df.columns if "id_" in c]
info_columns = [
    "speaker_id",
    "face_id",
    "gender",
    "split",
    "video_id",
    "clip_id",
    "batch",
    "audio_path"
]

test_df["dataset_id"] = test_df.apply(
    lambda row: f"{row['speaker_id']}_{row['video_id']}_{row['clip_id']}", 
    axis=1
)
test_df = test_df.set_index("dataset_id")
test_df["audio_path"] = test_df.apply(
    lambda row: os.path.join(PROJECT_PATH, "audio", f"{row['speaker_id']}_{row['video_id']}_{row['clip_id']}.wav"), 
    axis=1
)

In [ ]:
MAX_TEST_FILES = 500

X_test = np.array([load_audio(path) for path in test_df["audio_path"].head(MAX_TEST_FILES)])
X_test = X_test.reshape(-1, NUM_SAMPLES, 1)

y_test = test_df[ID_COLUMN_NAMES].head(MAX_TEST_FILES).values

In [ ]:
torch.cuda.is_available()

In [ ]:
len(tf.config.experimental.list_physical_devices("GPU")) > 0

In [ ]:
if (len(tf.config.experimental.list_physical_devices("GPU")) > 0):
    print("GPU is available!")
    gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.75)
    sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [ ]:
MODEL_NAME = "stft_vgg16_fixed-29.11.09.37.14-1.1.keras"


model = tf.keras.models.load_model(os.path.join(PROJECT_PATH, "models", MODEL_NAME))

In [ ]:
test_predictions = model.predict(X_test, verbose=1)

In [ ]:
bfmGenerator = BFMGenerator(bfm_folder = os.path.join(PROJECT_PATH, "face_geometry", "bfm"))

In [ ]:
index = random.randint(0, len(y_test) - 1)

y_test_tensor = torch.from_numpy(y_test[index]).float()
test_predictions_tensor = torch.from_numpy(test_predictions[index]).float()
bfmGenerator.plot_comparison(
    id_coeffs_list=[
        y_test_tensor, test_predictions_tensor
    ],
    titles=[
        "Target Face", "Predicted Face"
    ],
    plot_type="plotly"
)